In [ ]:
import sys, os
import setGPU
import hls4ml
import numpy as np
import h5py
import pickle

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.layers import Input, Dense, Lambda, BatchNormalization, Activation, Concatenate, Dropout, Layer
from tensorflow.keras.layers import ReLU, LeakyReLU, Reshape
from tensorflow.keras import backend as K

import tensorflow_model_optimization as tfmot
from qkeras import QDense, QActivation

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

from custom_layers import KLLoss, Sampling, Radius, CustomMSE
from functions import load_model

hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation', 'KLLoss']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND_CONV'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

In [ ]:
# load dataset
# Data = (N,19,3,1).flatten()
with open('/eos/user/e/epuljak/forDelphes/Delphes_QCD_BSM_data.pkl', 'rb') as f:
    X_train_flatten, X_train_scaled, X_test_flatten, X_test_scaled, bsm_data, bsm_target, pt_scaler = pickle.load(f)

In [ ]:
with open('/eos/user/e/epuljak/autoencoder_models/test_data.pkl', 'rb') as f:
#     data = [X_test_flatten, bsm_data, pt_scaler]
#     pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
    X_test_flatten, bsm_data, pt_scaler = pickle.load(f)

In [ ]:
# define do you want to create KL layer (False if it's standard autoencoder)
KL_layer = False
# define do you want to create MSE layer (False if it's variational autoencoder)
MSE_layer = True

In [ ]:
# load chosen model
custom_objects={'QDense': QDense, 'QActivation': QActivation, 'Sampling': Sampling}

#final_encoder = load_model('VAE_models/final_models/VAE_encoder_qkeras8_qinput', custom_objects=custom_objects)
#final_decoder = load_model('VAE_models/final_models/VAE_decoder_qkeras8_qinput', custom_objects=custom_objects)

autoencoder = load_model('/eos/user/e/epuljak/autoencoder_models/AE_qkeras8_notprunedw', custom_objects=custom_objects)

In [ ]:
model = autoencoder
model.summary()

In [ ]:
# check quantizers
for layer in model.layers:
    if hasattr(layer, "kernel_quantizer"):
        print(layer.name, "kernel:", str(layer.kernel_quantizer_internal), "bias:", str(layer.bias_quantizer_internal))
    elif hasattr(layer, "quantizer"):
        print(layer.name, "quantizer:", str(layer.quantizer))

In [ ]:
# if it's encoder
#pred_mean, pred_logvar, _ = model.predict(X_test_flatten[:500])

In [ ]:
# check if kl loss has correct values
# from losses import kl_loss
# print(pred_mean)
# print(pred_logvar)
# print('KL LOSS: ', kl_loss(pred_mean, pred_logvar))
# print('sigma: ', np.sqrt(np.exp(pred_logvar)))

In [ ]:
# get mu and sigma from model
if KL_layer:
    z_mean = final_encoder.layers[-3].output
    z_log_var = final_encoder.layers[-2].output
    # calculate KL distance with the custom layer
    custom_output = KLLoss()([z_mean, z_log_var])
    # create new model
    custom_model = Model(inputs=final_encoder.input, outputs=custom_output)
    custom_model.summary()

In [ ]:
if MSE_layer:
    true = model.input
    predicted = model.layers[-1].output
    reshaped_predicted = Reshape((19,3,1), name='reshaped_predicted')(predicted)

    scaled_input = BatchNormalization(trainable=False, name='scaled_input')(true)
    reshaped_scaled_input = Reshape((19,3,1), name='reshaped_scaled_input')(scaled_input)
    # calculate MSE between them
    custom_output = CustomMSE()([reshaped_scaled_input, reshaped_predicted])
    # create new model
    custom_model = Model(inputs=model.input, outputs=custom_output)
    """ Keras BatchNorm layer returns
        gamma * (batch - self.moving_mean) / sqrt(self.moving_var + epsilon) + beta
        epsilon=0.001
        momentum=0.99
        moving_mean = moving_mean * momentum + mean(batch) * (1 - momentum)
        moving_variance = moving_var * momentum + var(batch) * (1 - momentum)
        pt_scaler
        pt_scaler.mean_
        pt_scaler.var_
    """
    # with open('output/data_-1.pickle', 'rb') as f:
    #     x_train, y_train, _, _, _, pt_scaler = pickle.load(f)
    mean_ = np.zeros((57,))
    var_ = np.ones((57,))
    for i in range(19):
        mean_[3*i] = pt_scaler.mean_[i]
        var_[3*i] = pt_scaler.var_[i]
    # order of weights is (gamma,beta,mean,std)
    custom_model.get_layer('scaled_input').set_weights((np.ones((57,)),np.zeros((57,)),mean_,var_))
    custom_model.summary()
    custom_model.compile()

## Convert to HLS

In [ ]:
def update_config(config):
    #config['Model']['Strategy'] = 'Resource'
    config['LayerName']['input_1'].update({
        'Precision': 'ap_fixed<16,10>'
        })
    
#     config['LayerName']['kl_loss'].update({
#                 'Precision': {
#                     'accum': 'ap_fixed<32,16,AP_RND,AP_SAT>',
#                     'result': 'ap_fixed<32,16>'
#                 },
#                 'sum_t': 'ap_fixed<32,16>',
#                 'exp_table_t': 'ap_fixed<32,16,AP_RND,AP_SAT>',
#                 'table_size': 1024*4
#             })
    config['LayerName']['custom_mse']['Precision'].update({
            'result': 'ap_fixed<16, 10, AP_RND_CONV, AP_SAT>'
        })
    return config

In [ ]:
hardware = 'xcvu9p-flgb2104-2-e'

In [ ]:
config = hls4ml.utils.config_from_keras_model(custom_model, 
                                              default_precision='ap_fixed<16,6,AP_RND_CONV,AP_SAT>',
                                                max_bits=20,
                                                data_type_mode='auto_accum', # auto_accum_only
                                                granularity='name')

In [ ]:
config = update_config(config)

In [ ]:
from hls4ml.utils.config import set_accum_from_keras_model
set_accum_from_keras_model(config, custom_model)

In [ ]:
hls_model = hls4ml.converters.convert_from_keras_model(custom_model,
                                                           hls_config=config,
                                                           output_dir='output/dense_AE/xcvu9p/',
                                                           fpga_part=hardware)

In [ ]:
import pickle
with open('/eos/user/e/epuljak/autoencoder_models/config_AE_HLS.pickle', 'wb') as handle:
    pickle.dump(config, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# tu zapne!
hls4ml.model.profiling.numerical(model=custom_model, hls_model=hls_model, X=X_test_flatten[:10000])
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file='AE_HLS.pdf')

In [ ]:
hls4ml.model.profiling.compare(keras_model, hls_model, X_test_flatten[:1000000], 'dist_diff')

## Trace and inspect

In [ ]:
for layer in config['LayerName'].keys():
    config['LayerName'][layer]['Trace'] = True
hls_model = hls4ml.converters.convert_from_keras_model(custom_model,
                                                       hls_config=config,
                                                       output_dir='output/dense/xcvu9p/',
                                                       fpga_part=hardware)

In [ ]:
# take hls model predictions
hls_model.compile()
y_hls = hls_model.predict(X_test_flatten[:10000])

In [ ]:
# take keras model predictions
keras_model = custom_model
keras_model.compile()
# predictions keras model
y = keras_model.predict(X_test_flatten[:10000])

In [ ]:
# trace
hls4ml_pred, hls4ml_trace = hls_model.trace(X_test_flatten[:10000])

keras_trace = hls4ml.model.profiling.get_ymodel_keras(custom_model, X_test_flatten[:10000])

In [ ]:
# check tracing
kl_loss_total = []
for layer in keras_trace.keys():
    if layer in hls4ml_trace.keys():
        print(f'Keras layer {layer}, first sample:')
        print(config['LayerName'][layer])
        print(keras_trace[layer][1].flatten()-hls4ml_trace[layer][1].flatten())
        print("========================================================================")

In [ ]:
# new_keys = list()
# for k in list(keras_trace.keys()):
#     for h in list(hls4ml_trace.keys()):
#         if k == h and k not in new_keys and not k.startswith('batch') : new_keys.append(k)
#     if k.startswith('batch'): new_keys.append(k)

In [ ]:
# check output layer by layer
for layer in new_keys:
    print("================================================")
    if layer.startswith('batch_normalization_'):
        print("Keras layer %s, first sample:"%layer)
        print(keras_trace[layer][0])
        
        print("hls4ml layer %s, first sample:"%layer)
        print(hls4ml_trace[layer][0])
    else:
        print("Keras layer %s, first sample:"%layer)
        print(keras_trace[layer][0])
        print("hls4ml layer %s, first sample:"%layer)
        print(hls4ml_trace[layer][0])

In [ ]:
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True,
        to_file=f'plots/hls-model.pdf')

### Check ROCs for Keras vs HLS model

In [ ]:
y_hls = hls_model.predict(X_test_flatten[:2000000])
y_keras = keras_model.predict(X_test_flatten[:2000000])

In [ ]:
# checking if the results are the same without KL layer as output

# for layer in keras_trace.keys():
#     if layer in hls4ml_trace.keys():
#         if layer == 'q_dense_2': 
#             mean_qcd_keras = keras_trace[layer]
#             mean_qcd_hls = hls4ml_trace[layer]
#         if layer == 'q_dense_3': 
#             logvar_qcd_keras = keras_trace[layer]
#             logvar_qcd_hls = hls4ml_trace[layer]
#         print("========================================================================")

# from losses import kl_loss

# kl_loss_total = []
# kl_loss_total.append(kl_loss(mean_qcd_keras, logvar_qcd_keras))
# kl_loss_total.append(kl_loss(mean_qcd_hls, logvar_qcd_hls))

In [ ]:
loss_total = []
loss_total.append(y)
loss_total.append(y_hls)

In [ ]:
bsm_labels = ['Leptoquark','A to 4 leptons', 'hChToTauNu', 'hToTauTau']
labels = ['QCD keras', 'QCD hls',\
          r'keras LQ $\rightarrow$ b$\tau$', r'hls LQ $\rightarrow$ b$\tau$',\
          r'keras A $\rightarrow$ 4L', r'hls A $\rightarrow$ 4L',\
          r'keras $h_{\pm} \rightarrow \tau\nu$', r'hls $h_{\pm} \rightarrow \tau\nu$',\
          r'keras $h_{0} \rightarrow \tau\tau$', r'hls $h_{0} \rightarrow \tau\tau$']
loss = '$D_{KL}$'

In [ ]:
# checking if the results are the same without KL layer as output
# bsm_results = []

# for i, label in enumerate(bsm_labels):
#     hls4ml_pred, hls4ml_trace = hls_model.trace(bsm_data[i])
#     keras_trace = hls4ml.model.profiling.get_ymodel_keras(custom_model, bsm_data[i])
#     for layer in keras_trace.keys():
#         if layer in hls4ml_trace.keys():
#             if layer == 'q_dense_2': 
#                 mean_bsm_keras = keras_trace[layer]
#                 mean_bsm_hls = hls4ml_trace[layer]
#             if layer == 'q_dense_3': 
#                 logvar_bsm_keras = keras_trace[layer]
#                 logvar_bsm_hls = hls4ml_trace[layer]
#     kl_loss_total.append(kl_loss(mean_bsm_keras, logvar_bsm_keras))
#     kl_loss_total.append(kl_loss(mean_bsm_hls, logvar_bsm_hls))
#     print("========================================================================")

In [ ]:
for i, label in enumerate(bsm_labels):
    hls4ml_pred = hls_model.predict(bsm_data[i])
    keras_pred = keras_model.predict(bsm_data[i])
    
    loss_total.append(keras_pred)
    loss_total.append(hls4ml_pred)
    print("========================================================================")

In [ ]:
# output_result = 'AE_result_HLS.h5'

In [ ]:
# h5f = h5py.File(output_result, 'w')
# h5f.create_dataset('QCD', data = X_test_scaled)
# h5f.create_dataset('QCD_input', data=X_test_flatten)
# h5f.create_dataset('predicted_QCD', data = qcd_pred)
# for i, bsm in enumerate(bsm_results):
#     h5f.create_dataset('%s_scaled' %bsm[0], data=bsm[1])
#     h5f.create_dataset('%s_input' %bsm[0], data=bsm_data[i])
#     h5f.create_dataset('predicted_%s' %bsm[0], data=bsm[2])

# h5f.close()

In [ ]:
minScore = 999999.
maxScore = 0
for i in range(len(labels)):
    thisMin = np.min(loss_total[i])
    thisMax = np.max(loss_total[i])
    minScore = min(thisMin, minScore)
    maxScore = max(maxScore, thisMax)

In [ ]:
colors = ['C1','C2', 'C3', 'C4', 'C5', 'C6']


In [ ]:
bin_size=100
plt.figure(figsize=(10,8))
z = 0
for i, label in enumerate(labels):
    if i%2==0:
        plt.hist(loss_total[i].reshape(loss_total[i].shape[0]*1), bins=bin_size, label=label, density = True, range=(minScore, maxScore),
         histtype='step', fill=False, linewidth=1.5, color=colors[z])
    if i%2==1:
        plt.hist(loss_total[i].reshape(loss_total[i].shape[0]*1), bins=bin_size, label=label, density = True, range=(minScore, maxScore),
         histtype='step', fill=False, linewidth=1.5, alpha=0.6, color=colors[z])
        z = z+1
#plt.semilogx()
plt.semilogy()
plt.xlabel("Loss")
plt.ylabel("Probability (a.u.)")
plt.grid(True)
plt.title('KL loss')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
tpr_lq=[];fpr_lq=[];auc_lq=[]
tpr_ato4l=[];fpr_ato4l=[];auc_ato4l=[]
tpr_ch=[];fpr_ch=[];auc_ch=[]
tpr_to=[];fpr_to=[];auc_to=[]


target_qcd = np.zeros(loss_total[0].shape[0])
target_qcd_hls = np.zeros(loss_total[1].shape[0])

for i, label in enumerate(labels):
    if i == 0 and i==1: continue
    if i%2==0:
        trueVal = np.concatenate((np.ones(loss_total[i].shape[0]), target_qcd))
        predVal_loss = np.concatenate((loss_total[i], loss_total[0]))

        fpr_loss, tpr_loss, threshold_loss = roc_curve(trueVal, predVal_loss)

        auc_loss = auc(fpr_loss, tpr_loss)
        if i==2:
            tpr_lq.append(tpr_loss)
            fpr_lq.append(fpr_loss)
            auc_lq.append(auc_loss)
        elif i == 4:
            tpr_ato4l.append(tpr_loss)
            fpr_ato4l.append(fpr_loss)
            auc_ato4l.append(auc_loss)
        elif i==6:
            tpr_ch.append(tpr_loss)
            fpr_ch.append(fpr_loss)
            auc_ch.append(auc_loss)
        elif i == 8:
            tpr_to.append(tpr_loss)
            fpr_to.append(fpr_loss)
            auc_to.append(auc_loss)
    if i%2==1:
        trueVal = np.concatenate((np.ones(loss_total[i].shape[0]), target_qcd_hls))
        predVal_loss = np.concatenate((loss_total[i], loss_total[1]))

        fpr_loss, tpr_loss, threshold_loss = roc_curve(trueVal, predVal_loss)

        auc_loss = auc(fpr_loss, tpr_loss)
        if i==3:
            tpr_lq.append(tpr_loss)
            fpr_lq.append(fpr_loss)
            auc_lq.append(auc_loss)
        elif i == 5:
            tpr_ato4l.append(tpr_loss)
            fpr_ato4l.append(fpr_loss)
            auc_ato4l.append(auc_loss)
        elif i==7:
            tpr_ch.append(tpr_loss)
            fpr_ch.append(fpr_loss)
            auc_ch.append(auc_loss)
        elif i == 9:
            tpr_to.append(tpr_loss)
            fpr_to.append(fpr_loss)
            auc_to.append(auc_loss)

In [ ]:
plt.figure(figsize=(12,8))
for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_lq[:], fpr_lq[:], auc_lq[:], labels[2:4])):
    if i == 1:
        plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[0], alpha=0.6)
    else: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[0])

for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_ato4l[:], fpr_ato4l[:], auc_ato4l[:], labels[4:6])):
    if i == 1: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[1], alpha = 0.6)
    else: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[1])
for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_ch[:], fpr_ch[:], auc_ch[:], labels[6:8])):
    if i==1: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[2], alpha=0.6)
    else: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[2])

for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_to[:], fpr_to[:], auc_to[:], labels[8:])):
    if i==1: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[3], alpha=0.6)
    else: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[3])
plt.semilogx()
plt.semilogy()
plt.ylabel("True Positive Rate", fontsize=15)
plt.xlabel("False Positive Rate", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True)
plt.legend(bbox_to_anchor=[1.2, 0.5],loc='best',frameon=True)
plt.tight_layout()
plt.plot(np.linspace(0, 1),np.linspace(0, 1), '--', color='0.75')
plt.axvline(0.00001, color='red', linestyle='dashed', linewidth=1)
plt.show()